# **`sagesim` Implementation**

`SAGESim` (Scalable Agent-Based GPU-Enabled Simulator) is the first scalable, pure-Python, general-purpose agent-based modeling framework that supports both distributed computing and GPU acceleration. It is designed for use on the latest generation of high-performance computing systems. This series of notebooks will guide you through using **`sagesim`** to simulate large-scale agent-based models (ABMs) tailored to your specific problem domain. The core workflow involves defining a custom model class that subclasses the base `Model` class provided by `sagesim`. This enables access to the built-in `simulate()` method to execute your simulations.

Before building your model, it’s important to understand how agent data is organized and transferred across MPI ranks. In `sagesim`, agent data on each MPI rank is structured into **Agent Data Tensors (ADTs)** — CuPy-based vectors that store property values for all agents managed by that rank, optimized for GPU execution.

To construct your own model class, there are three main components you’ll need to implement:

#### 1. **Define and Register Breeds**

Each agent in your model must belong to a specific *breed*. To enable this:

- Define a breed class by subclassing the `Breed` class from `sagesim`. Each breed class is responsible for:
   - Registering agent properties using `breed.register_property()` for each property.
   - Defining the **step functions**, which specifies how agents behave at each simulation step, and registering it using `breed.register_step_func()`. Note that each breed can have multiple step functions, each assigned a different execution priority.
- Register the breed inside the model’s `__init__()` method using `model.register_breed()`.

#### 2. **Define and Register the Reduce Function**

`sagesim` supports parallel execution by distributing agents across multiple compute nodes and/or GPUs. After executing the simulation steps, different copies of agent data might exists, that is multiple version of adts. Therefore, a function is called to handles the logical combination (or "reduction") of those copies into a single consistent version. To enable this:

- Define the problem specific *reduce function*
- Register the *reduce function* in the `__init__()` method using `register_reduce_function()`.

This function is critical for ensuring logical correctness in distributed simulations.


#### 3. **Create and Connect Agents**

Define separate class methods that

- Specify which breed each agent belongs to and assign initial values for the agent's attributes using `create_agent_of_breed()`.
- Connect two agents, using `connect_agents()`.


#### Others: 
- If you have any global properties, they should be registered in the model class's `__init__()` method using `register_global_property()`.


#### CuPy Implementation: What It Means to You

`sagesim` uses a **CuPy** implementation to support both NVIDIA CUDA and AMD ROCm GPUs. However, there are important constraints when using **`cupyx.jit.rawkernel`**. Kernel code must be written using low-level Python functions, as many advanced Python features and abstractions are not supported. 

As a result, when implementing your own `step functions` and `reduce functions`, you must adhere to these limitations. Key restrictions include (but are not limited to):

- NaN checks must be done via inequality to self (e.g., `x != x`). This is an unfortunate limitation of `cupyx`.
- Dictionaries and custom Python objects are not supported.
- `*args` and `**kwargs` are unsupported.
- Nested function definitions are not allowed.
- Use **CuPy** data types and array routines instead of NumPy: [https://docs.cupy.dev/en/stable/reference/routines.html](https://docs.cupy.dev/en/stable/reference/routines.html)
- `for` loops must use the `range` iterator only — no `for-each` style loops.
- `return` statements do not behave reliably.
- `break` and `continue` statements are unsupported.
- Variables cannot be reassigned within `if` or `for` blocks. Declare and assign them at the top level or within new subscopes.
- Negative indexing (e.g., `array[-1]`) may not work as expected; it can access memory outside the logical bounds of the array. Use `len(array) - 1` instead.





# **SIR-ABM Model**

In this notebook, we demonstrate how to implement an agent-based model (ABM) of the **Susceptible–Infected–Recovered (SIR)** framework using the **`sagesim`** library.

The **SIR model** is a foundational tool in epidemiology, used to describe how infectious diseases spread through a population. By adopting an **agent-based approach**, we simulate this process at the individual level, capturing the complexity and heterogeneity of real-world interactions that drive transmission.

By the end of this notebook, you will have a fully functional **SIR-ABM simulator** that highlights how individual behaviors influence disease dynamic -- along with a practical understanding of how to apply **`sagesim`** to your own modeling tasks.


We implement an ABM where:

- Each **agent** represents an individual with unique characteristics, particularly their **preventative measures**, which influence their response to exposure. Agents can be in one of three possible states:
  - **Susceptible**: Healthy but vulnerable to infection upon contact with an infected agent.
  - **Infected**: Currently carrying the disease and capable of infecting susceptible agents through interactions.
  - **Recovered**: No longer infected and assumed immune; they cannot be reinfected.


- Disease transmission occurs through interactions between *connected agents*—that is, agents capable of coming into contact with each other. Importantly, the probability that a susceptible agent becomes infected is **not uniform** across the population. It depends on the agent's individual **preventative behaviors**. Agents with stronger preventative measures are **less likely** to become infected upon exposure. Examples of such **preventative behaviors** include:
  - Hygiene practices (e.g., handwashing, mask-wearing)
  - Social distancing
  - Vaccination status


For the remainder of this notebook, we will:

1. **Define the `SIRModel` class**  
   Create a subclass of the `Model` class from the `sagesim` library tailored to the SIR-ABM framework.

2. **Instantiate the SIR model**  
   Construct an instance of `SIRModel` using a generated *Watts–Strogatz* small-world network.

3. **Run the simulation and analyze results**  
   Execute the simulation on the model instance and collect results to analyze the dynamics of disease spread.



## Define the `SIRModel` class ##

### 1. **Define and Register `SIRBreed`**

As explained earlier, we begin by defining a custom breed by subclassing the `Breed` class from the `sagesim` library. In the SIR model, we use a single breed to represent the general population. This breed is implemented as the `SIRBreed` class.

- Each agent in this breed has two primary properties:

    - **`state`**: A categorical variable indicating the agent's infection status. We encode the three SIR states as:
        - `1` — Susceptible  
        - `2` — Infected  
        - `3` — Recovered

    - **`preventative_measures`**: A list of 100 floating-point values that capture the agent’s individual behavior traits—such as hygiene practices, social distancing adherence, or vaccination status—that influence their likelihood of infection.


- Recall we register step functions with specified priority levels to define how agents of a breed behave at each simulation step. In the case of the SIR model, we use a single step function that governs how an agent's state evolves over time—for instance, determining whether a susceptible agent becomes infected or an infected agent recovers.

### 

In [ ]:
from enum import Enum
from time import time
from random import random, sample
import networkx as nx


# import the Breed class from sagesim
from sagesim.breed import Breed

# Define the SIRState enumeration for agent states
class SIRState(Enum):
    SUSCEPTIBLE = 1
    INFECTED = 2
    RECOVERED = 3

# Define the step function to be registered for SIRBreed
def step_func(agent_ids, agent_index, globals, breeds, locations, states, preventative_measures):
    """
    At each simulation step, this function evaluates a subset of agents—either all agents in a serial run or a partition assigned to
    a specific rank in parallel processing—and determines whether an agent's state should change based on interactions with its neighbors
    and their respective preventative behaviors.

    Parameters:
    ----------
    agent_ids : list[int]
        List of all agent IDs in the simulation.
    agent_index : int
        Index of the agents --> please check, I am not sure how this is used
    globals : list
        Global parameters; the first global parameter is always [please fill in], followed by user defined global parameters. 
    breeds : list
        List of breed objects (unused here as we only have one type of breed, but must passed for interface compatibility).
    locations : list[list[int]]
        Adjacency list specifying neighbors for each agent.
    states : list[int]
        List of current state of each agent.
        This is a user specified input, must match the order of the properties will be defined in the breed's __init__ method.
        In this case, the state property be defined first here, followed by the list of preventative measure property.
    preventative_measures : list[list[float]]
        List of vectors representing each agent’s preventative behaviors. 
        Again, this is a user specified input, must match the order of the properties will be defined in the breed __init__ method.

    Returns:
    -------
    None
        The function updates the `states` list in-place if an agent becomes infected.
    """

    # Retrieve this agent’s neighbors
    neighbor_ids = locations[agent_index]

    # Skip step if the agent is not susceptible, i.e., if it is already infected or recovered.
    if int(states[agent_index]) != 1:
        return

    # Get global infection probability
    p_infection = globals[1]

    # Preventative measures of the current (susceptible) agent
    agent_pm = preventative_measures[agent_index]

    # Draw a random number for probabilistic infection check
    rand_val = random()

    # Loop over all neighbors
    for neighbor_id in neighbor_ids:
        # Find index of neighbor in agent_ids list
        try:
            neighbor_index = agent_ids.index(neighbor_id)
        except ValueError:
            continue  # skip if neighbor ID not found (should not happen)

        # Check if the neighbor is infected
        if int(states[neighbor_index]) == 2:
            neighbor_pm = preventative_measures[neighbor_index]

            # Compute interaction risk based on both agents' preventative behaviors
            interaction_risk = 0.0
            for a in agent_pm:
                for b in neighbor_pm:
                    interaction_risk += a * b

            # Normalize by vector length squared
            max_risk = len(agent_pm) ** 2
            normalized_risk = interaction_risk / max_risk

            # Infection probability depends on 1 - joint safety
            if rand_val < p_infection * (1 - normalized_risk):
                states[agent_index] = 2  # Agent becomes infected
                return  # No need to continue checking other neighbors


In [ ]:
# now we define the SIRBreed class, which inherits from the Breed class
class SIRBreed(Breed):
    """
    SIRBreed class the SIR model.
    Inherits from the Breed class in the sagesim library.
    """

    def __init__(self) -> None:
        name = "SIR"
        super().__init__(name) 
        # Register properties for the breed
        self.register_property("state", SIRState.SUSCEPTIBLE.value) 
        self.register_property("preventative_measures", [-1 for _ in range(100)])
        # Register the step function
        self.register_step_func(step_func)


### 2. **Define and Register the Reduce Function**

The reduce function for the SIR model handles merging the states of agents from different copies from different MPI ranks (in case of parallel execution or distributed processing). The function works as follows:

- If one copy of an agent is infected (state 2) and the other copy is susceptible (state 1), the agent will be considered infected.
- If one copy of an agent is recovered (state 3) and the other is either susceptible (state 1) or infected (state 2), the agent will be considered recovered

Thus, the reduce function simply compares the state values from both copies and assigns the maximum value as the state of the agents.


In [ ]:
def reduce_agent_data_tensors_(adts_A, adts_B):
    """
    This function takes two agent data tensors (adts_A and adts_B) and reduces them into a single tensor.
    Parameters:
    ----------
    adts_A : list
        The first agent data tensor.
    adts_B : list
        The second agent data tensor.
    Returns:    
    -------
    list
        The reduced agent data tensor.
    """
    
    result = []
    # breed would be same as first
    result.append(adts_A[0])
    # network would be same as first
    result.append(adts_A[1])
    # state would be max of both
    result.append(max(adts_A[2], adts_B[2]))
    return result


### 3. **Create and Connect Agents**

With the `SIRBreed` and the reduction function `reduce_agent_data_tensors_` defined, we're now ready to initialize the `SIRModel`. The next step is to implement a class method that creates agents and establishes connections between them.

- **Creating agents**: Use the model method `create_agent_of_breed()`, which takes the breed object along with user-defined breed properties (such as `state` and `preventative_measures`). It returns the unique ID of the newly created agent.

- **Connecting agents**: Use `self.get_space().connect_agents()` to connect two agents by their IDs. This establishes a neighbor relationship between them in the simulation space.

This model also includes a **global property**: the base infection probability. This represents the baseline probability that a susceptible agent becomes infected when in contact with an infected neighbor. The final infection probability is adjusted based on the `preventative_measures` characteristics of both the agent and its neighbor.

With these components in place, we’re ready to define the `SIRModel` class tailored to simulate the dynamics of the SIR agent-based model.

In [ ]:
from sagesim.model import Model
from sagesim.space import NetworkSpace # hopefully, we can avoid this import


class SIRModel(Model):
    """
    SIRModel class for the SIR model.
    Inherits from the Model class in the sagesim library.
    """

    def __init__(self, p_infection=1.0) -> None:
        space = NetworkSpace()
        super().__init__(space)
        self._sir_breed = SIRBreed()
        self.register_breed(breed=self._sir_breed)
        self.register_global_property("p_infection", p_infection)
        self.register_reduce_function(reduce_agent_data_tensors_)

    def create_agent(self, state, preventative_measures):
        agent_id = self.create_agent_of_breed(
            self._sir_breed, state=state, preventative_measures=preventative_measures
        )
        self.get_space().add_agent(agent_id)
        return agent_id

    def connect_agents(self, agent_0, agent_1):
        self.get_space().connect_agents(agent_0, agent_1)


## **Instantiate the SIR Model**  

For this notebook, we use the *Watts–Strogatz* small-world network for mimic of real-world contact networks, this graph captures key features of real-world contact networks, such as high clustering and short path lengths, making it well-suited for modeling infectious disease spread. Each node in the graph will correspond to an agent, and edges will define neighbor relationships used for potential transmission. 

Once the network is created, pass it to the model's constructor along with any required parameters (e.g., initial infection probability, number of agents, etc.).

Create a SIR model with agents in a small-world network topology.

Parameters:
- model (SIRModel): An instance of the SIR model.
- num_agents (int): Total number of agents.
- num_init_connections (int): Each agent is connected to this many neighbors.
- num_infected (int): Number of initially infected agents.

Returns:
- SIRModel: Initialized model with agents and connections.




In [ ]:
model = SIRModel()





# MPI environment setup
comm = MPI.COMM_WORLD
num_workers = comm.Get_size()
worker = comm.Get_rank()

num_agents = 1000
num_init_connections = 6
num_nodes = 1  # Logical nodes, can be used for partitioning


def generate_small_world_network(n, k, p):
    """
    Generate a small-world network using the Watts-Strogatz model.

    Parameters:
    - n (int): Number of nodes (agents).
    - k (int): Each node is connected to its k nearest neighbors.
    - p (float): Probability of rewiring an edge (introduces randomness).

    Returns:
    - networkx.Graph: Generated network.
    """
    return nx.watts_strogatz_graph(n, k, p)


network = generate_small_world_network(num_agents, num_init_connections, 0.2)

for n in network.nodes:
    preventative_measures = [random() for _ in range(100)]
    model.create_agent(SIRState.SUSCEPTIBLE.value, preventative_measures)

for n in sample(sorted(network.nodes), num_infected):
    model.set_agent_property_value(n, "state", SIRState.INFECTED.value)

for edge in network.edges:
    model.connect_agents(edge[0], edge[1])


## 👥 Create and Initialize Agents
Agents are placed in the network and randomly assigned an initial state.


In this simple SIR model, we define only **one breed**, representing the general population. 

## ⚙️ Define Simulation Parameters
You can modify these values to run different configurations.

In [14]:
num_agents = 1000
num_init_connections = 6
num_nodes = 1  # Logical nodes, can be used for partitioning

## 🛠️ Set up the SIR Model

In [ ]:
model = SIRModel()
model.setup(use_gpu=True)  # Enables GPU acceleration if available

AttributeError: 'SIRModel' object has no attribute 'register_reduce_function'

## 🧱 Build the Agent Network

In [ ]:
model_creation_start = time()

model = generate_small_world_of_agents(
    model,
    num_agents,
    num_init_connections,
    int(0.1 * num_agents),  # 10% initially infected
)

model_creation_end = time()
model_creation_duration = model_creation_end - model_creation_start
print(f"Model creation took {model_creation_duration:.2f} seconds.")

## ▶️ Run the Simulation

In [ ]:
simulate_start = time()

model.simulate(num_ticks=10, sync_workers_every_n_ticks=1)

simulate_end = time()
simulate_duration = simulate_end - simulate_start
print(f"Simulation took {simulate_duration:.2f} seconds.")

## 📊 Collect Final States

In [ ]:
if worker == 0:
    result = [
        SIRState(model.get_agent_property_value(agent_id, property_name="state"))
        for agent_id in range(num_agents)
        if model.get_agent_property_value(agent_id, property_name="state") is not None
    ]
    print(f"Final state distribution: {[str(state) for state in result[:10]]}...")

## 💾 Save Execution Metrics

In [ ]:
import csv

if worker == 0:
    with open("execution_times.csv", "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([
            num_agents,
            num_init_connections,
            num_nodes,
            num_workers,
            model_creation_duration,
            simulate_duration
        ])
    print("Execution time written to 'execution_times.csv'.")